<a href="https://colab.research.google.com/github/MirieleCampos/AnaliseDeConfiabilidade/blob/main/Reliability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assets

In [4]:
from tabulate import tabulate      # print tables
import numpy as np

def showReliabilitySystemData(reliabilityData,  name):
  tab_text = [["Component", "Length (km)", "Failure rate 𝝀 (failure/year)", "Mean Time to repair r (h)"]]
  for k in range( np.shape(reliabilityData)[0] ):
    tab_text.append(
        [(reliabilityData[k][0]),
         "{:.2f}".format(reliabilityData[k][1]),
         "{:.2f}".format(reliabilityData[k][2]),
         "{:.2f}".format(reliabilityData[k][3])
         ] )
  print(name)
  print(tabulate(tab_text))

def showLoadSystemData(loadSystemData,  name):
  tab_text = [["Name", "Circuit node", "Consumers", "Load (kW)"]]
  for k in range( np.shape(loadSystemData)[0] ):
    tab_text.append(
        [(loadSystemData[k][0]),
         "{:.0f}".format(loadSystemData[k][1]),
         "{:.0f}".format(loadSystemData[k][2]),
         "{:.0f}".format(loadSystemData[k][3])
        ] )
  print(name)
  print(tabulate(tab_text))

def showLoadSystemIndexes(loadSystemIndexes, name):
  #   0      1       2         3     4   5  6
  #["name", node, consumers, load, lamb, U, r]
  tab_text = [["Name Load", "Circuit node", "N_consumers", "Load (kW)", "𝜆", "U", "r"]]
  for kL in range( np.shape(loadSystemIndexes)[0] ):
    tab_text.append([(
          loadSystemIndexes[kL][0]),
          "{:.0f}".format(loadSystemIndexes[kL][1]),
          "{:.0f}".format(loadSystemIndexes[kL][2]),
          "{:.0f}".format(loadSystemIndexes[kL][3]),
          "{:.3f}".format(loadSystemIndexes[kL][4]),
          "{:.3f}".format(loadSystemIndexes[kL][5]),
          "{:.3f}".format(loadSystemIndexes[kL][6])
        ] )
  print(name)
  print(tabulate(tab_text))

def calcIndexes(loadSystemIndexes):
  N_vet = np.array([line[2] for line in loadSystemIndexes])
  L_vet = np.array([line[3] for line in loadSystemIndexes])
  y_vet = np.array([line[4] for line in loadSystemIndexes])
  U_vet = np.array([line[5] for line in loadSystemIndexes])
  r_vet = np.array([line[6] for line in loadSystemIndexes])
  indexes = {
      'SAIFI': "{:.3f}".format( sum(y_vet*N_vet)/sum(N_vet) ),
      'SAIDI': "{:.3f}".format(sum(U_vet*N_vet)/sum(N_vet)  ),
      #'CAIFI': "Not defined",
      'CAIDI': "{:.3f}".format(sum(U_vet*N_vet)/sum(y_vet*N_vet) ),
      'ASAI' : "{:.6f}".format((sum(N_vet*8760) - sum(U_vet*N_vet))/sum(N_vet*8760) ),
      'ASUI' : "{:.6f}".format(1 - (sum(N_vet*8760) - sum(U_vet*N_vet))/sum(N_vet*8760) ),
      'ENS'  : "{:.3f}".format(sum(L_vet*U_vet) ),
      'AENS' : "{:.2f}".format(sum(L_vet*U_vet)/sum(N_vet) ),
  }
  return indexes

def showSystemIndexes(loadSystemIndexes, indexes, name):
  tab_text = [["Index", "Value"]]
  indexes_matrix = [[key, value] for key, value in indexes.items()]
  for line in indexes_matrix:
    tab_text.append( line )
  print(name)
  print(tabulate(tab_text))
  return indexes

def getAdjacencyAndRangeMatrix(fromToData):
  n_nodes = np.max(fromToData)
  n_lines = np.shape(fromToData)[0]

  # Adjacency Matrix
  A = np.zeros((n_nodes, n_nodes))
  for line in range( n_lines ):
    node_from   = fromToData[line][0] - 1
    node_to     = fromToData[line][1] - 1
    A[node_from][node_to] = 1

  # Range Matrix
  A = A.astype(int)
  I = np.eye(n_nodes)
  R_aux = np.linalg.matrix_power(A + I, n_nodes - 1)
  R = (R_aux >= 1).astype(int)
  return A, R

# Summary





1. Leitura de dados;
2. Definir a matriz de alcance para análise dos instrumentos de proteção;
3. Inicializar variáveis vetoriais 𝑼𝒑𝒐𝒏𝒕𝒐 e 𝝀𝒑𝒐𝒏𝒕𝒐, onde a posição indica a indisponibilidade e taxa de falha de cada ponto, respectivamente;
4. Definir o caso a ser avaliado;
5. Para cada componente/linha:
 - Buscar a taxa de falha 𝜆𝐿𝑇 do componente na matriz de dados;
   
	 Se caso 1:  
	•  Buscar protetor na matriz de dados;
	•  Buscar o vetor de indicadores de barras desconectadas pela falta bd através da matriz de alcance;
	•  Buscar o tempo médio de reparo  𝑟𝐿𝑇  do componente na matriz de dados;
	•  Calcular a indisponibilidade  𝑈𝐿𝑇 = 𝜆𝐿𝑇×𝑟𝐿𝑇 , e atualizar indicadores de ponto como segue:
      𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝜆𝐿𝑇
		  𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝑈𝐿𝑇

	 Se caso 2:
	•  Buscar protetor na matriz de dados;
	•  Buscar o vetor de indicadores de barras desconectadas pela falta bd através da matriz de alcance;
	•  Buscar o tempo médio de reparo  𝑟𝐿𝑇  do componente na matriz de dados;
	•  Calcular a indisponibilidade  𝑈𝐿𝑇=𝜆𝐿𝑇×𝑟𝐿𝑇, e atualizar indicadores de ponto como segue:
	    𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅×𝜆𝐿𝑇
		  𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅×𝑈𝐿𝑇

	 Se caso 3:
	• Buscar protetor na matriz de dados;
	• Buscar o vetor de indicadores de barras desconectadas pela falta bd
	através da matriz de alcance;

			• Se a falha não implica em abertura da seccionadora:
						a. Buscar o tempo médio de reparo 𝑟𝐿𝑇 do componente;
						b. Calcular a indisponibilidade 𝑈𝐿𝑇 = 𝜆𝐿𝑇 × 𝑟𝐿𝑇 e atualizar
						indicadores de ponto como segue:
								𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝜆𝐿𝑇
								𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝑈𝐿𝑇

			• Se a falha implica em abertura da seccionadora:
					a. Buscar o vetor de barras desconectadas pela seccionadora bds
					através da matriz de alcance;
					b. Buscar o tempo médio de reparo 𝑟𝐿𝑇 do componente;
					c. Calcular a indisponibilidade 𝑈𝐿𝑇 = 𝜆𝐿𝑇 × 𝑟𝐿𝑇;
					d. Busca r referente à seccionadora (𝑟𝑠𝑒𝑐 = 0,5 ℎ𝑜𝑟𝑎𝑠);
					e. Calcular a indisponibilidade 𝑈𝑠𝑒𝑐 = 𝜆𝐿𝑇 × 𝑟𝑠𝑒𝑐;
					f. Atualizar os indicadores de cada ponto usando as equações;
							𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝜆𝐿𝑇
							𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅𝒔 × 𝑈𝐿𝑇 + (𝒃𝒅 − 𝒃𝒅𝒔) ×𝑈𝑠𝑒c

	 Se caso 4:
	• Buscar protetor na matriz de dados;
	• Buscar o vetor de barras desconectadas bd pela falta através da matriz
	de alcance;

			• Se a falha não implica em acionamento da seccionadora e da alimentação alternativa:
					a. Buscar o tempo médio de reparo rLT do componente;
					b. Calcular a indisponibilidade 𝑈𝐿𝑇 = 𝜆𝐿𝑇 × 𝑟𝐿𝑇 e atualizar indicadores de ponto;
							𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝜆𝐿𝑇
  						𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝑈𝐿𝑇

			• Se a falha implica em acionamento da seccionadora e da alimentação alternativa:
					a. Se existe abertura da seccionadora: Buscar as barras desconectadas pela seccionadora bds através da matriz de alcance; caso contrário, bds é vetor de zeros;
					b. Se acionamento da alimentação alternativa: Buscar as barras bda que serão supridas pela alimentação alternativa através da matriz de alcance; caso não haja acionamento da alimentação alternativa: bda é vetor de zeros;
					c. Buscar o tempo médio de reparo rLT do componente;
					d. Calcular a indisponibilidade 𝑈𝐿𝑇 = 𝜆𝐿𝑇 × 𝑟𝐿𝑇;
					e. Busca r referente à seccionadora (𝑟𝑠𝑒𝑐 = 0,5 ℎ𝑜𝑟𝑎𝑠);
					f. Calcular a indisponibilidade 𝑈𝑠𝑒𝑐 = 𝜆𝐿𝑇 × 𝑟𝑠𝑒𝑐;
					g. Busca r referente à alimentação alternativa (𝑟𝑎𝑙𝑖𝑚_𝑎𝑙𝑡 = 0,5 ℎ𝑜𝑟𝑎𝑠);
					h. Calcular a indisponibilidade 𝑈𝑎𝑙𝑖𝑚_𝑎𝑙𝑡 = 𝜆𝐿𝑇 ×𝑟𝑎𝑙𝑖𝑚_𝑎𝑙𝑡;
					i. Atualizar valores de cada ponto com sua contribuição;
								𝝀𝒑𝒐𝒏𝒕𝒐 = 𝝀𝒑𝒐𝒏𝒕𝒐 + 𝒃𝒅 × 𝜆𝐿𝑇
								𝑼𝒑𝒐𝒏𝒕𝒐 = 𝑼𝒑𝒐𝒏𝒕𝒐 + (𝒃𝒅 − 𝒃𝒅𝒔) × 𝑈𝑠𝑒𝑐 + (𝒃𝒅𝒔 −𝒃𝒅𝒂) × 𝑈𝐿𝑇 + 𝒃𝒅𝒂 × 𝑈𝑎𝑙𝑖𝑚_𝑎𝑙𝑡

6. Aplicar equações (1)-(14) para o cálculo dos indicadores sistêmicos SAIFI, SAIDI, CAIDI, ASAI, ASUI, ENS e AENS.

# Code

## 1. Reading data

In [5]:
# Reading data
import numpy as np
from tabulate import tabulate      # print tabelas

# ["Component", "Length (km)", "Failure rate 𝝀 (failure/year)", "Mean Time to repair r (h)"]
reliabilitySystemData = [
  ["Section 1", 2, 0.2, 4],
  ["Section 2", 1, 0.1, 4],
  ["Section 3", 3, 0.3, 4],
  ["Section 4", 2, 0.2, 4],
  ["Distrib. a",1, 0.2, 2],
  ["Distrib. b",3, 0.6, 2],
  ["Distrib. c",2, 0.4, 2],
  ["Distrib. d",1, 0.2, 2],
]

# ["Name", "Circuit node", "Consumers", "Load (kW)"]
loadSystemData = [
    ["A", 6, 1000, 5000],
    ["B", 7, 800, 4000],
    ["C", 8, 700, 3000],
    ["D", 9, 500, 2000],
]
# Montando Matriz de adjacencia
fromToData = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [2, 6],
    [3, 7],
    [4, 8],
    [5, 9]
])

dataTableName = "Parâmetros de Confiabilidade do sistema -  Bilington, 1991 - Pág 234"
showReliabilitySystemData(reliabilitySystemData,  dataTableName)

dataTableName = " Número de Consumidores e Carga Conectada ao sistema - Bilington, 1991 - Pág 234"
showLoadSystemData(loadSystemData,  dataTableName)

## 2. Getting Adjacency and Range Matrices
A, R = getAdjacencyAndRangeMatrix(fromToData)
#print(A)
#print(R)

# [ "component", "case 1",  "case 2", "case 3", "case 4"]


protectorComponent = [
  ["Section 1",  1, 1, 1, 1],
  ["Section 2",  1, 1, 1, 1],
  ["Section 3",  1, 1, 1, 1],
  ["Section 4",  1, 1, 1, 1],
  ["Distrib. a", 1, 6, 6, 6],
  ["Distrib. b", 1, 7, 7, 7],
  ["Distrib. c", 1, 8, 8, 8],
  ["Distrib. d", 1, 9, 9, 9],
]
'''
Busca na Matriz de alcance o protetor do componente,
analisando o nó onde o protetor está associado.
Linha da Matriz de alcance: Protetor do componente
coluna da Matriz de alcance: Consumidor de análise.
'''

seccionadora_bds = [
  ["Section 1", "","", 0, 2],
  ["Section 2", "","", 3, 3],
  ["Section 3", "","", 4, 4],
  ["Section 4", "","", 5, 5],
  ["Distrib. a", "","",0, 0],
  ["Distrib. b", "","",0, 0],
  ["Distrib. c", "","",0, 0],
  ["Distrib. d", "","",0, 0],
]

'''
Analisa se houve uma falha no componente (sessões e distribuidores):
Se a falha não implica em abertura da seccionadora:
  Se seccionadora_bds = 0. Não ha seccionadora que isole a falha.
Se a falha implica em abertura da seccionadora:
    Buscar as barras desconectadas pela seccionadora bds.
    Se seccionadora_bds != 0: nó desconectado no reestabelecimento.
'''


alimentadora_bda = [
  ["Section 1", "","","", 3],
  ["Section 2", "","","", 4],
  ["Section 3", "","","", 5],
  ["Section 4", "","","", 0],
  ["Distrib. a", "","","",0],
  ["Distrib. b", "","","",0],
  ["Distrib. c", "","","",0],
  ["Distrib. d", "","","",0],
]

'''
Analisa se a falha implica em acionamento da seccionadora
e da alimentação alternativa:
Se existe abertura da seccionadora:
  Buscar as barras desconectadas pela seccionadora bds,  caso contrário, bds = 0.
Se acionamento da alimentação alternativa:
  Buscar as barras bda que serão supridas pela alimentação alternativa, caso contrário bda = 0.
'''



Parâmetros de Confiabilidade do sistema -  Bilington, 1991 - Pág 234
----------  -----------  -----------------------------  -------------------------
Component   Length (km)  Failure rate 𝝀 (failure/year)  Mean Time to repair r (h)
Section 1   2.00         0.20                           4.00
Section 2   1.00         0.10                           4.00
Section 3   3.00         0.30                           4.00
Section 4   2.00         0.20                           4.00
Distrib. a  1.00         0.20                           2.00
Distrib. b  3.00         0.60                           2.00
Distrib. c  2.00         0.40                           2.00
Distrib. d  1.00         0.20                           2.00
----------  -----------  -----------------------------  -------------------------
 Número de Consumidores e Carga Conectada ao sistema - Bilington, 1991 - Pág 234
----  ------------  ---------  ---------
Name  Circuit node  Consumers  Load (kW)
A     6             1000       500

'\nAnalisa se a falha implica em acionamento da seccionadora \ne da alimentação alternativa:\nSe existe abertura da seccionadora:\n  Buscar as barras desconectadas pela seccionadora bds,  caso contrário, bds = 0.\nSe acionamento da alimentação alternativa:\n  Buscar as barras bda que serão supridas pela alimentação alternativa, caso contrário bda = 0.\n'

## 3. Algorithm

In [6]:
# Definir o caso a ser avaliado
caseToAccess = int(input("Definir o caso a ser avaliado: "))

# Inicializar variáveis vetoriais 𝑼𝒑𝒐𝒏𝒕𝒐 e 𝝀𝒑𝒐𝒏𝒕𝒐,
# em que a posição indica a indisponibilidade e taxa de falha de cada ponto,
# respectivamente;

#   0      1       2         3     4   5  6
#["name", node, consumers, load, lamb, U, r]
loadSystemIndexes = []
for k in range( len(loadSystemData) ):
  loadSystemIndexes.append([
      loadSystemData[k][0],
      loadSystemData[k][1],
      loadSystemData[k][2],
      loadSystemData[k][3],
      0, 0, 0])

# Para cada componente/linha:
n_components = len(reliabilitySystemData)
for kLT in range( n_components ):
 # print("Olhando componente: {}".format(reliabilitySystemData[kLT][0]))
  # Buscar a taxa de falha 𝜆𝐿𝑇 do componente na matriz de dados;
  yLT = reliabilitySystemData[kLT][2]
  # Buscar o tempo médio de reparo 𝑟𝐿𝑇 do componente na matriz de dados;
  rLT = reliabilitySystemData[kLT][3]
  # Calcular a indisponibilidade 𝑈𝐿𝑇 = 𝜆𝐿𝑇×𝑟𝐿𝑇 , e atualizar
  ULT = yLT*rLT

  # Buscar protetor na matriz de dados;
  pLT = protectorComponent[kLT][caseToAccess]

  idx_seccionadora     = seccionadora_bds[kLT][caseToAccess]
  idx_alimentadora_bda = alimentadora_bda[kLT][caseToAccess]

  # Para cada ponto de carga
  for kLoad in range( len(loadSystemIndexes) ):

    # Nó da carga
    node_load = loadSystemIndexes[kLoad][1]
  #  print("Olhando componente: {}".format(node_load))

    bd = R[pLT-1][node_load-1]
    #print("bd = {} para (pLT, Load) ({},{})".format(db, pLT, node_load))


#condição para caso 1:
    if caseToAccess == 1:
      lambLoad = loadSystemIndexes[kLoad][4]
      ULoad    = loadSystemIndexes[kLoad][5]

      lambLoad = lambLoad + bd*yLT
      ULoad    = ULoad    + bd*ULT

      loadSystemIndexes[kLoad][4] = lambLoad
      loadSystemIndexes[kLoad][5] = ULoad


#condição para caso 2:
    if caseToAccess == 2:
      lambLoad = loadSystemIndexes[kLoad][4]
      ULoad    = loadSystemIndexes[kLoad][5]

      lambLoad = lambLoad + bd*yLT
      ULoad    = ULoad    + bd*ULT

      loadSystemIndexes[kLoad][4] = lambLoad
      loadSystemIndexes[kLoad][5] = ULoad

#condição para caso 3:
    if caseToAccess == 3:

      if idx_seccionadora == 0:   #Falha não implica em abertura da seccionadora
          lambLoad = loadSystemIndexes[kLoad][4]
          ULoad    = loadSystemIndexes[kLoad][5]

          lambLoad = lambLoad + bd*yLT
          ULoad    = ULoad    + bd*ULT

          loadSystemIndexes[kLoad][4] = lambLoad
          loadSystemIndexes[kLoad][5] = ULoad

      if idx_seccionadora != 0:  # Falha implica em abertura da seccionadora
          Usec = yLT * 0.5  # Indisponibilidade da seccionadora
          bd = R[pLT-1][node_load-1]
          bds = R[idx_seccionadora-1][node_load-1]

          lambLoad = loadSystemIndexes[kLoad][4]
          ULoad    = loadSystemIndexes[kLoad][5]

          lambLoad = lambLoad + bd*yLT
          ULoad    = ULoad    + bds*ULT + (bd - bds)*Usec

          loadSystemIndexes[kLoad][4] = lambLoad
          loadSystemIndexes[kLoad][5] = ULoad

# condição para caso 4:
    if caseToAccess == 4: # se caso 4
      # Se a Falha não implica em abertura da seccionadora e alimentação alternativa
      if (idx_seccionadora == 0):
        lambLoad = loadSystemIndexes[kLoad][4]
        ULoad    = loadSystemIndexes[kLoad][5]

        lambLoad = lambLoad + bd*yLT
        ULoad    = ULoad    + bd*ULT

        loadSystemIndexes[kLoad][4] = lambLoad
        loadSystemIndexes[kLoad][5] = ULoad

      # Se a Falha implica em abertura da seccionadora e alimentação alternativa
      if (idx_seccionadora != 0): # abertura da seccionadora
        # a) Se existe abertura da seccionadora:
        bds = R[idx_seccionadora-1][node_load-1]

        # b) se acionamento da alimentação alternativa:
        if (idx_alimentadora_bda == 0): # sem alimentação alternativa
          bda = 0
        else: # com alimentação alternativa
          bda = R[idx_alimentadora_bda-1][node_load-1]

        # cálculos
        Usec = yLT * 0.5  # Indisponibilidade da seccionadora
        Ualim = yLT *0.5
        lambLoad = loadSystemIndexes[kLoad][4]
        ULoad    = loadSystemIndexes[kLoad][5]

        lambLoad = lambLoad + bd*yLT
        ULoad    = ULoad  + (bd - bds)*Usec + (bds - bda)*ULT + (bda)*Ualim

        loadSystemIndexes[kLoad][4] = lambLoad
        loadSystemIndexes[kLoad][5] = ULoad


for kLoad in range( len(loadSystemIndexes) ):
  lambLoad = loadSystemIndexes[kLoad][4]
  ULoad    = loadSystemIndexes[kLoad][5]
  loadSystemIndexes[kLoad][6] = ULoad/lambLoad

name = "LoadSystemIndexes p/ Caso {}".format(caseToAccess)
showLoadSystemIndexes(loadSystemIndexes, name)

indexes = calcIndexes(loadSystemIndexes)
name = "SystemIndexes p/ Caso {}".format(caseToAccess)
indexes = showSystemIndexes(loadSystemIndexes, indexes, name)



Definir o caso a ser avaliado: 4
LoadSystemIndexes p/ Caso 4
---------  ------------  -----------  ---------  -----  -----  -----
Name Load  Circuit node  N_consumers  Load (kW)  𝜆      U      r
A          6             1000         5000       1.000  1.500  1.500
B          7             800          4000       1.400  1.950  1.393
C          8             700          3000       1.200  2.250  1.875
D          9             500          2000       1.000  1.500  1.500
---------  ------------  -----------  ---------  -----  -----  -----
SystemIndexes p/ Caso 4
-----  ---------
Index  Value
SAIFI  1.153
SAIDI  1.795
CAIDI  1.556
ASAI   0.999795
ASUI   0.000205
ENS    25050.000
AENS   8.35
-----  ---------
